# Project - Group 15
Jennifer, Irene, Zac, Yi <br>
Stat 159

In [23]:
import pandas as pd
import numpy as np

## Data

### Import Datasets

#### ToDo: Add descriptions for dataset source

In [10]:
# Opioid Overdose Deaths
opioid = pd.read_csv('data/Multiple Cause of Death 1999-2014 v1.1.csv')
opioid

,State,Year,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Prescriptions Dispensed by US Retailers in that year (millions)
0,Alabama,1999,39,4430141,0.9,0.6,1.2,116
1,Alabama,2000,46,4447100,1,0.8,1.4,126
2,Alabama,2001,67,4467634,1.5,1.2,1.9,138
3,Alabama,2002,75,4480089,1.7,1.3,2.1,142
4,Alabama,2003,54,4503491,1.2,0.9,1.6,149
...,...,...,...,...,...,...,...,...
811,Wyoming,2010,49,563626,8.7,6.4,11.5,210
812,Wyoming,2011,47,568158,8.3,6.1,11,219
813,Wyoming,2012,47,576412,8.2,6,10.8,217
814,Wyoming,2013,52,582658,8.9,6.7,11.7,207


In [41]:
# State economic data 2010-2014
gdp = pd.read_csv('data/Energy Census and Economic Data US 2010-2014.csv')
gdp.head()

,StateCodes,State,Region,Division,Coast,Great Lakes,TotalC2010,TotalC2011,TotalC2012,TotalC2013,...,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014
0,AL,Alabama,3.0,6.0,1.0,0.0,1931522,1905207,1879716,1919365,...,1.165832,1.157861,-0.020443,-0.168414,0.396416,0.420102,1.011941,1.001333,1.562247,1.577963
1,AK,Alaska,4.0,9.0,1.0,0.0,653221,653637,649341,621107,...,3.203618,2.869760,-1.175137,-1.949571,-3.789313,-13.754494,0.948185,1.835376,-0.585695,-10.884734
2,AZ,Arizona,4.0,8.0,0.0,0.0,1383531,1424944,1395839,1414383,...,1.090035,1.091283,1.341472,-0.420875,-0.580562,-1.313050,2.317801,0.621971,0.509473,-0.221767
3,AR,Arkansas,3.0,7.0,0.0,0.0,1120632,1122544,1067642,1096438,...,2.141877,2.129805,1.369514,5.131282,3.910476,6.280636,3.336628,7.155212,6.052353,8.410441
4,CA,California,4.0,9.0,1.0,0.0,7760629,7777115,7564063,7665241,...,4.207353,4.177389,-1.162079,-1.173951,-1.341226,-0.830982,2.761377,2.772770,2.866127,3.346406


### Data Cleaning

In [37]:
# drop rows with suppressed, unreliable, or na values
opioid = opioid[opioid['Deaths']!='Suppressed']
opioid = opioid[opioid['Crude Rate']!='Unreliable']
gdp = gdp.dropna()

### Data Preprocessing
#### ToDo: may turn this cell into reproducible tools

In [38]:
# convert quantitative data to numeric data type
opioid['Deaths'] = pd.to_numeric(opioid['Deaths'])
opioid['Crude Rate'] = pd.to_numeric(opioid['Crude Rate'])

### Data Engineering

In [40]:
# select opioid data within 2010-2014 (range of gdp data)
df = opioid[(opioid['Year']<=2014) & (opioid['Year']>=2010)]

# group data by year and state; there's only 
df = df.groupby(['Year', 'State']).mean()
df

/tmp/ipykernel_215/2283414156.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['Year', 'State']).mean()


Deaths  Population  Crude Rate  \
Year State                                           
2010 Alabama         194.0   4779736.0         4.1   
     Alaska           71.0    710231.0        10.0   
     Arizona         650.0   6392017.0        10.2   
     Arkansas        198.0   2915918.0         6.8   
     California     2059.0  37253956.0         5.5   
...                    ...         ...         ...   
2014 Virginia        775.0   8326289.0         9.3   
     Washington      692.0   7061530.0         9.8   
     West Virginia   585.0   1850326.0        31.6   
     Wisconsin       640.0   5757564.0        11.1   
     Wyoming          57.0    584153.0         9.8   

                    Prescriptions Dispensed by US Retailers in that year (millions)  
Year State                                                                           
2010 Alabama                                                    210.0                
     Alaska                                                     210.0                
     Arizona                                                    210.0                
     Arkansas                                                   210.0                
     California                                                 210.0                
...                                                               ...                
2014 Virginia                                                   196.0                
     Washington                                                 196.0                
     West Virginia                                              196.0                
     Wisconsin                                                  196.0                
     Wyoming                                                    196.0                

[251 rows x 4 columns]

In [ ]:
# join two datasets together based on year and state
#df.merge(gdp, left_index = True, right_on = ['Year'])

## Author Contributions
Jennifer: 
<br>
Irene:
<br>
Zac: 
<br>
Yi: 